# Report project Basics of mobile robotics
Members: Mahdi Fourati, Yahya Hadi, Fatih Yilmaz, Alex Zanetta

## Introduction

This Jupyter Notebook serves as a final report for the project of Basics of mobile robotics. We will first go over our initial choices, then make a deep dive into the specific parts of the code.

### Initial choices
* Environment: we decided to use a simple blue floor with black obstacles and a red objective to have a strong contrast between them and the robot to make the vision easier. The obstacles can have any shape, which make the math heavier but more representative of the reality.
* Global navigation: the path from the starting point to the goal is computed using Dijkstra's algorithm. We decided to use Dijkstra's algorithm to allow us more flexibility in case we wanted to further complexify the problem and add some intermediary goals (which doesn't work with heuristics based algorithm such as A*). The points that the algorithm uses are the vertices of the obstacles approximated a bit more than half the robot's width away to avoid crashing into them while taking the uncertainty into account. 
* Pose estimation : the filter we used to estimate the pose and to control the robot is an extended Kalman filter. We chose it because all the prerequisite for its use where fullfilled and it is the de facto standard in navigation systems [[1]](#1).
* Motion control: we used a standard P-lead controller to make the robot follow the trajectory. As the trajectory is linear and the odometry reasonably good (the same speed on the two wheels will make the robot go straight with very little error), we don't need to make it more complex and use a PID controller or any othere more complex.
* Local navigation: this part makes the robot avoid crashing into obstacles if it senses something right in front of him. The shape of the obstacles is not relevant: the robot will move around it in the direction where it didn't sense anything. If all the sensors sense something, i.e. the robot is in a corner, it will do a 180°.




## Vision

### Getting the field of play

To get the field of play (FoP), we begin by getting the four corners of the blue rectangle on the raw video feed, and approximate it by a four corners polygon. In order to find the corners, we threshold the input image on green and red, look for the intersection between and invert it. The operation of threshholding takes an image with a single colour channel (by opposition to the standard 3 colours RGB format), and sets all pixel below a threshold parameter to 0 (black) and all above to 255 (white):

$$
out(x,y)=
\begin{cases}
255 & \quad \text{when $src(x,y) > parameter$}\\ 
0 & \quad \text{otherwise}
\end{cases}
$$

We use a thresh parameter low enough to get the whole border but high enough to avoid getting the FoP. Once we have done the thresholding on the two colour channel green and red we make a bitwise AND on them:

$$
out(x,y)=
\begin{cases}
255 & \quad \text{when $thresh_{green}(x,y) \neq 0 \: \& \: thresh_{red}(x,y) \neq 0$}\\ 
0 & \quad \text{otherwise}
\end{cases}
$$

Finally, we invert the result. This yields the FoP as a white shape on a black background. This allows us to find the contours of the shape. They are all the boundary points of a white object on a black background. We make the computations more efficient by throwing away all unnecessary points (e.g. we don't need all points of a straight line, just the first and the last). This is the code doing this part:  

---
```r
_, thresholded_green = cv.threshold(copy[:,:,1],127,255,cv.THRESH_BINARY)
_, thresholded_red = cv.threshold(copy[:,:,2],127,255,cv.THRESH_BINARY)

thresholded = thresholded_green & thresholded_red
thresholded = (255 - thresholded)

contours, _ = cv.findContours(thresholded, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
```
---



We chose to use a blue floor to make the contrasts as large as possible with the other elements. Since the camera is not perfectly over the FoP, the raw video feed yields a tilted FoP with borders possibly containing unwanted elements. We must then cast this raw FoP onto a perfect rectangle more representative of the reality. To do this, we use a perspective transformation. We begin by approximating the contour by a four sided polygon using a Douglas-Peucker algorithm, which recursively divide the original contour until the variation between the newly computed contour and the original one is the one specified[[2]](#2). Once we have our 4 vertices, we compute the matrix casting them onto a perfect rectangle of width and height same as the maximum width and height of the polygon. 

INSERT MATH

This transformation is a combination of a similarity, a shear transformation, a scaling and an elation that can be formulated that way [[3]](#3): 

$$ 
out(x,y) =
\begin{bmatrix}
s \times cos(\theta) & -s \times sin(\theta) & t_{x} \\
s \times sin(\theta) & s \times cos(\theta) & t_{y} \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & k & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\lambda & 0 & 0 \\
0 & 1/\lambda & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
v_{1} & v_{2} & v
\end{bmatrix}
$$

These are the code snippets highlighting the main aformentioned points:

---
```r
# approximate the field of play to a polygon with 4 vertices
epsilon = 0.05 # 5% delta between contour and approximation
perimeter = cv.arcLength(c, True) # True means closed contour
approximation = cv.approxPolyDP(c, epsilon * perimeter, True)

# do the perspective transformation
matrix = cv.getPerspectiveTransform(original_coordinates, new_coordinates)
fop = cv.warpPerspective(frame, matrix, (int(new_coordinates[3][0]), int(new_coordinates[3][1])))
```
---

### Getting the robot starting position

It is quite easy to get the location of thymio with our set up. We simply look at the white zone on the FoP. To do this, we grayscale it to get the average BGR pixel intensity and threshholds on a high value. What is more complicated is getting the starting angle of the robot. This is done by looking for the curved part of the robot. We make two different polygonal approximations of the robot, both precise but with slightly different precisions. We then use a bitwise XOR to get the slight difference in curvature and compute the center of this difference. Once we have it, we compute the vector from the center of the robot to the center of the difference and get the complex argument of this vector, interpreting the y coordinate as the imaginary part. The argument is computed that way[[4]](#4):

$$
Arg(x,y)=
\begin{cases}
arctan(\dfrac{y}{x}) & \quad \text{when $x > 0$}\\ 
arctan(\dfrac{y}{x}) + \pi & \quad \text{when $x < 0 \: \& \: y \geq 0$}\\ 
arctan(\dfrac{y}{x}) - \pi & \quad \text{when $x < 0 \: \& \: y < 0$}\\ 
\dfrac{\pi}{2} & \quad \text{when $x = 0 \: \& \: y > 0$}\\ 
-\dfrac{\pi}{2} & \quad \text{when $x = 0 \: \& \: y < 0$}\\ 
undefined & \quad \text{when $x = 0 \: \& \: y = 0$}
\end{cases}
$$

### Getting the obstacles

The contours of the obstacles are obtained in a similar way than the FoP and the starting position of the robot, the math behind it has already been explained in said sections. What has not yet been explained is how we get the vertices placed futher away from the obstacles to avoid the robot crashing in them.

## Global planning
 
In order to make our robot move from his starting position to the goal, we had to find the shortest path using all the vercities found by the vision to avoid any obstacle. To accomplish this feature, we are using a dijkstra algorithm. 

### Mapping

From the vision, we get an array of points representing each verticities of all obstacles, the starting position of the robot and the coordinates of the goal.

Since we have all the points in the map, we can name them and start looking for connexions between them. 
- Are they from the same obstacle ? 
- Are they connected ? 
- What distance separetes them ? 


To answer those questions, we used python dictionnaries and some useful insights from the web. 
For example, to check if two points are connected, we had to check if the straight line between them doesn't intersect with any line formed by two adjacent verticities of a same obstacle(instersect with an obstacle).
To figure out if two lines intersects, we used the concept of orientation explained in the vision part METTRE REF ICI. 

For the distance, we used for the entirity of the global_planning the euclidian distance. 
This is the formula that we used: 
$$ 
d(p_1, p_2) = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}
$$

Now that we have dicctionaries of connected points and the distances between them, we can go on and start finding the shortest path to move the robot from its starting position to the goal. 




### Dijkstra algorithm

We chose to use this algorithm for our project because it is fairly simple to implement in addition to being very flexible (e.g not needing a grid) and the documentation on it is abundant compared to other algorithms.

In order to implement this famous algorithm, we took as template the algorithm found on this webpage : METTRE LA REF ICI https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html
This is also what inspired us the use of dictionnaries in the first place. 

Basically, the algorithm works by getting the graph (in our case a dictionnary of adgencency + dictionnary of the coordinates of points in order to calculate the distance between them). Having done that, we compute the best path for the robot by checking every node starting with the robot, mesuring and comparing distances between node and keeping the nodes that produces the shortest path. Finally, by calling the function "global_plan" we get the shortest path staring from the robot, avoiding obstacles until it hits the goal. 




## Sources

[1]: <a id="1">Wikipedia, “Extended Kalman filter” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Extended_Kalman_filter</a>

[2]: <a id="2">Wikipedia, “Ramer–Douglas–Peucker algorithm” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Ramer%E2%80%93Douglas%E2%80%93Peucker_algorithm</a>

[3]: <a id="3">Medium, “Part II: Projective Transformations in 2D” [Last access 26 November 2024]. Available: https://medium.com/@unifyai/part-ii-projective-transformations-in-2d-2e99ac9c7e9f</a>

[4]: <a id="4">Wikipedia, “Argument (complex analysis)” [Last access 25 November 2024]. Available: https://en.wikipedia.org/wiki/Argument_(complex_analysis)</a>
